# How to make your Python programs faster?
### by Dhruv Baldawa (@dhruvbaldawa)

## Disclaimer

```html
This talk was previously named "Competition Programming and Problem solving using Python".
There might still be some remnants from that talk but I have updated it to be
more up-to-date and add other relevant information

# @TODO(DB): find a better description
```

## Why do we run into problems?

 - Big dataset
 - Great complexity
 - Python interpreter limitations
 - All of the above

## What this talk covers?

 - Memory management
   - lists v/s iterables
   - itertools _(my favorite part)_
 - Time management
   - correct data structures
   - approach
 - New advancements
   - multiprocessing
   - Alternative interpreters
   - Alternative compilers

## Memory management

### Why is important?

 - Most of the competitions require you store and compute results from large datasets.	
 - Running out of memory limit is pretty common.
 - There is "always" a way to improve memory consumption of your program.
 - You can always throw more hardware, but fixing it in the software just feels nice ;).

### Common problems

 - **MemoryError**[1]
   - when you run out of memory but still you can get out of it (by deleting few objects)
 - **OverflowError**[1]
   - when the results of an arithmetic operation is too large to be represented

### List forms

 - better to store and re-use results of computations	
 - use if you want to perform list operations, where you need to store all the items	
 - consumes	much more memory
 - only prefer when a real list cannot be avoided
 - all the elements are generated and initialized at once

### Iterables

 - lazy and on-demand generation of values
 - very low memory consumption
 - very useful when you only need to work with the value at hand
 - **Cons**:
   - cannot step backwards 
   - cannot or jump forwards
 - they are scalable and memory-friendly and used where real lists are not required

In [1]:
%load_ext memory_profiler
BIG_NUMBER = 10**7

## range v/s xrange

#### range

In [2]:
%%memit
# range
sum_ = 0
l = range(BIG_NUMBER)
for x in l:
    sum_ += x

peak memory: 341.66 MiB, increment: 310.68 MiB


In [3]:
# Access Operations
print l[-1] # Pass

# Assignment operation
l[10] = -1 # Pass
print l[10]

9999999
-1


#### xrange

In [4]:
%%memit
# xrange
sum_ = 0
l = xrange(BIG_NUMBER)
for x in l:
    sum_ += x

peak memory: 342.02 MiB, increment: 0.00 MiB


In [5]:
# Access Operations
print l[-1] # Pass

# Assignment operation
# l[10] = -1 # FAILS !!
print l[10]

9999999
10


 - In Python 3, the default behavior for `range` is like `xrange` in Python 2
 - To get `range`-like behavior in Python 3, use `list(range(x, y))` instead

## List Comprehensions v/s Generator Expressions

### List Comprehensions

**Syntax**:
```python
[expression for variables in iterable if condition]
```

In [6]:
%%timeit
# Using loops
square_list = []
for x in range(BIG_NUMBER):
    square_list.append(x**2)

1 loops, best of 3: 3.11 s per loop


In [7]:
%%timeit
square_list = []
# List comprehensions
square_list = [x**2 for x in range(BIG_NUMBER)]
print square_list[:10]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
1 loops, best of 3: 1.6 s per loop



### Generator Expressions

**Syntax**:
```python
(expression for variables in iterable if condition)
```

In [8]:
%%timeit
# Generator Expressions
square_list = (x**2 for x in range(BIG_NUMBER))
print square_list.next()

0
0
0
0
1 loops, best of 3: 213 ms per loop


## Functions v/s Generators

#### Functions

```python
def function(params):
    ....
    ....
    return something
```

In [9]:
import math
def get_me_factorials_till(n):
    '''Returns a list of factorials from 1 to n'''
    return_list = []
    for x in range(1, n):
        return_list.append(math.factorial(x))
    return return_list

In [10]:
%memit
factorials = get_me_factorials_till(10)
print factorials

peak memory: 500.69 MiB, increment: 0.03 MiB
[1, 2, 6, 24, 120, 720, 5040, 40320, 362880]


#### Generators

```python
def function(params):
    ...
    ...
    yield something
```

In [11]:
import math
def generate_me_factorials_till(n):
    '''Generates factorials from 1 to n'''
    for x in range(1, n):
        yield math.factorial(x)

factorials = generate_me_factorials_till(10)

In [12]:
print factorials

# Its an iterator
print factorials.next()

<generator object generate_me_factorials_till at 0x7f724ece4dc0>
1


In [13]:
%memit
# Create a new generator, since the earlier one will be used up
factorials = generate_me_factorials_till(10)
for factorial in factorials:
    print factorial,

peak memory: 500.74 MiB, increment: 0.03 MiB
1 2 6 24 120 720 5040 40320 362880


## Set and Dictionary Comprehensions

**Set Comprehension:**
```python
{value for value in expressions if conditions}
```

**Dict Comprehensions:**
```python
{key: value for key, value in expressionss if conditions}
```

In [14]:
# Set comprehension
print {k for k in 'ABCDEFEDCBA'}

# Dictionary Comprehension
print {k: k+1 for k in range(10)}

set(['A', 'C', 'B', 'E', 'D', 'F'])
{0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10}


## import itertools

 - itertools module provides a set of fast, memory efficient iterators
 - provides fast implementations for common jobs like product, permutations, combinations

<img src="itertools.png" />

In [15]:
# N-Queens Puzzle
# Reference: http://code.activestate.com/recipes/576647-eight-queens-six-lines/
from itertools import permutations

n = 4
cols = range(n)
for vec in permutations(cols):
    if (n == len(set(vec[i]+i for i in cols))
          == len(set(vec[i]-i for i in cols))):
        print vec

(1, 3, 0, 2)
(2, 0, 3, 1)


In [16]:
# run-length encoding
from itertools import groupby
for key, group in groupby("aaaabbccccddd"):
    print key, len(list(group)),

a 4 b 2 c 4 d 3


## Time optimizations

 - choosing the right data structure
 - choosing the right approach

```html
    "Bad Programmers worry about code. Good programmers worry about data structures and their relationships"
    -- Linus Torvalds
```

 - **Don’t re-invent the wheel.** Use `tuple`, `list`, `dict`, `sets`, as they are coded in `C` and hence are _FAST_
 - for membership tests use `dict`/`set` [O(1)] instead of `list`/`tuple` [O(n)]
 - use `collections`
 - Queue operations like `pop()`, `insert()` are better in `collections.deque` [O(1)] than `list`[O(n) for insertion at the start]
 - use `bisect`, `heapq` for sorted lists


## import collections

 - `deque`
 - `Counter`
 - `OrderedDict`
 - `defaultdict`

#### deque

In [17]:
import collections
# Creating a deque

d = collections.deque(['first', 'second', 'third', 'current last'])
print d
print '--'

deque(['first', 'second', 'third', 'current last'])
--


In [18]:
# right rotation
d.rotate()
print '>>> d.rotate()'
print d
print '--'

# left rotation
d.rotate(-1)
print '>>> d.rotate(-1)'
print d
print '--'

>>> d.rotate()
deque(['current last', 'first', 'second', 'third'])
--
>>> d.rotate(-1)
deque(['first', 'second', 'third', 'current last'])
--


In [19]:
# append from left side
d.appendleft("new first")
print '>>> d.appendleft("new first")'
print d
print '--'

# remove from the left side
d.popleft()
print '>>> d.popleft()'
print d
print '--'

>>> d.appendleft("new first")
deque(['new first', 'first', 'second', 'third', 'current last'])
--
>>> d.popleft()
deque(['first', 'second', 'third', 'current last'])
--


In [20]:
d.append("new last")
print '>>> d.append("new last")'
print d
print '--'

d.pop()
print '>>> d.pop()'
print d
print '--'

>>> d.append("new last")
deque(['first', 'second', 'third', 'current last', 'new last'])
--
>>> d.pop()
deque(['first', 'second', 'third', 'current last'])
--


#### Counter

In [21]:
# Playground
from collections import Counter
c = Counter(a=3, b=1)
d = Counter(a=1, b=2)

print c, d
print '--'

Counter({'a': 3, 'b': 1}) Counter({'b': 2, 'a': 1})
--


In [22]:
# add two counters together:  c[x] + d[x]
print '>>> c + d'
print c + d
print '--'

# subtract (keeping only positive counts)
print '>>> c - d'
print c - d
print '--'

>>> c + d
Counter({'a': 4, 'b': 3})
--
>>> c - d
Counter({'a': 2})
--


In [23]:
# intersection:  min(c[x], d[x])
print '>>> c & d'
print c & d
print '--'

# union:  max(c[x], d[x])
print '>>> c | d'
print c | d
print '--'

>>> c & d
Counter({'a': 1, 'b': 1})
--
>>> c | d
Counter({'a': 3, 'b': 2})
--


In [24]:
text = "I saw Susie sitting in a shoe shine shop."

import re
c = Counter(re.split(r"\W+", text, flags=re.IGNORECASE))
print c
print '--'

new_text = "Where she sits she shines, and where she shines she sits."
print '>>> c.update(new_text)'
c.update(re.split(r"\W+", new_text, flags=re.IGNORECASE))
print c
print '--'

print '>>> c.most_common(3)'
print c.most_common(3)
print '--'

Counter({'a': 1, 'shine': 1, 'Susie': 1, 'shop': 1, 'sitting': 1, 'I': 1, 'shoe': 1, 'in': 1, 'saw': 1, '': 1})
--
>>> c.update(new_text)
Counter({'she': 4, 'shines': 2, 'sits': 2, '': 2, 'a': 1, 'shine': 1, 'and': 1, 'Susie': 1, 'shop': 1, 'sitting': 1, 'I': 1, 'Where': 1, 'where': 1, 'shoe': 1, 'in': 1, 'saw': 1})
--
>>> c.most_common(3)
[('she', 4), ('shines', 2), ('sits', 2)]
--


#### defaultdict

In [25]:
from collections import defaultdict

# defaultdict takes a default_factory as an argument
# you can always subclass it to have it with your own
# default factory
default_dict = defaultdict(int)
print default_dict[10]

0


## Memoization

 - caching results from previous procedure calls and using them directly
 - it can only be memoized if a function called with same arguments, **always** return the same result

```python
def memoized_function(value):
    if value in cache:
        return cache[value]
    else:
        cache[value] = compute(value)
        return cache[value]
```

### Collatz Conjecture

```html
The following iterative sequence is defined for the set of positive integers:

n ->  n/2 (n is even)
n -> 3n + 1 (n is odd)

Using the rule above and starting with 13, we generate the following sequence:
13 -> 40 -> 20 -> 10 -> 5 -> 16 -> 8 -> 4 -> 2 -> 1

Which starting number, under one lakh, produces the longest chain?
```

In [26]:
# Iterative solution
def chain(i):
    '''Iterative approach to solve the chain problem'''
    c = [i]
    last_element = i

    while last_element > 1:
        if last_element % 2 == 0:
            last_element = last_element / 2
        else:
            last_element = 3 * last_element + 1
        c.append(last_element)
    return c

def the_simple_chain():
    start = 2
    limit = 100000
    max_length = 1
    max_num = 1
    for i in xrange(start, limit):
        c = chain(i)
        if len(c) > max_length:
            max_num = i
            max_length = len(c)

    return max_num

%timeit the_simple_chain()

1 loops, best of 3: 4.78 s per loop


In [27]:
# Recursive solution
def chain(n):
    """
    Recursive function for the above expression
    """
    if n == 1:
        return [1]
    else:
        if n % 2 == 0:
            return [n] + chain(n / 2)
        else:
            return [n] + chain(3 * n + 1)

def the_recursive_chain():
    start = 2
    limit = 100000
    max_length = 1
    max_num = 1

    for i in xrange(limit, start, -1):
        chain_list = chain(i)

        if len(chain_list) > max_length:
            max_length = len(chain_list)
            max_num = i

    return max_num

%timeit the_recursive_chain()

1 loops, best of 3: 12 s per loop


In [28]:
# Recursive solution
cache = {}
def chain(n):
    """
    Memoized recursive function for the above expression
    """
    if n in cache:
        return cache[n]
    if n == 1:
        cache[1] = [1]
        return cache[1]
    else:
        if n % 2 == 0:
            cache[n] = [n] + chain(n / 2)
            return cache[n]
        else:
            cache[n] = [n] + chain(3 * n + 1)
            return cache[n]

def the_recursive_chain():
    start = 2
    limit = 100000
    max_length = 1
    max_num = 1

    for i in xrange(limit, start, -1):
        chain_list = chain(i)

        if len(chain_list) > max_length:
            max_length = len(chain_list)
            max_num = i

    return max_num

%timeit the_recursive_chain()

The slowest run took 6.97 times longer than the fastest. This could mean that an intermediate result is being cached 
1 loops, best of 3: 120 ms per loop


## Multiplication Problem

Implement multiplication operation when no multiplication operator is available

**Naive Implementation**

2 * 3 = 2 + 2 + 2

In [29]:
# Naive Multiplication
def naive(a, b):
    x = a
    y = b
    z = 0
    while x > 0:
        z = z + y
        x = x - 1
    return z

%timeit naive(BIG_NUMBER, BIG_NUMBER)

1 loops, best of 3: 2.04 s per loop


#### Russian Peasant's Algorithm

 - Write each number at the head of a column.
 - Double the number in the first column, and halve the number in the second column.
   - If the number in the second column is odd, divide it by two and drop the remainder.
 - If the number in the second column is even, cross out that entire row.
 - Keep doubling, halving, and crossing out until the number in the second column is 1.
 - Add up the remaining numbers in the first column. The total is the product of your original numbers.


```html
Let's multiply 57 by 86 as an example:
Write each number at the head of a column.

57 	 86 
Double the number in the first column, and halve the number in the second column.

57 	   86 
114    43 
If the number in the second column is even, cross out that entire row.

57 XX   86 XX 
114 	43 
Keep doubling, halving, and crossing out until the number in the second column is 1.

57 XX   86 XX
114 	43 
228 	21 
456 XX  10 XX
912 	 5 
1824 XX  2 
3648 	 1 
Add up the remaining numbers in the first column.

  114 	43 
  228 	21 
  912 	 5 
+ 3648     1
--------------
  4902
```

In [30]:
# Russian Peasant's Algorithm for Multiplication
def russian(a, b):
    x = a
    y = b
    z = 0
    while x > 0:
        if x % 2 == 1:
            z = z + y
        y = y << 1
        x = x >> 1
    return z

%timeit russian(BIG_NUMBER, BIG_NUMBER)

100000 loops, best of 3: 10.4 µs per loop


In [31]:
# Recursive Russian Peasant's Algorithm
def rec_russian(a, b):
    if a == 0:
        return 0
    if a % 2 == 0:
        return 2 * rec_russian(a/2, b)
    return b + 2 * rec_russian((a-1)/2, b)

%timeit rec_russian(BIG_NUMBER, BIG_NUMBER)

100000 loops, best of 3: 14.2 µs per loop


## Other tools

 - multiprocessing
 - other interpreters
 - other compilers

### `multiprocessing`

 - Utilize multiple cores
 - Communicate between the processes by using a `Queue` or a `Pipe`
 - Implements traditional concurrency primitives like locks, events and semaphores
 - Processes can also share memory

In [32]:
from multiprocessing import Process, Queue

def f(q):
    q.put([42, None, 'hello'])

if __name__ == '__main__':
    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    print(q.get())    # prints "[42, None, 'hello']"
    p.join()

[42, None, 'hello']


In [33]:
from multiprocessing import Process, Pipe

def f(conn):
    conn.send([42, None, 'hello'])
    conn.close()

if __name__ == '__main__':
    parent_conn, child_conn = Pipe()
    p = Process(target=f, args=(child_conn,))
    p.start()
    print(parent_conn.recv())   # prints "[42, None, 'hello']"
    p.join()

[42, None, 'hello']


### PyPy

 - Works out of the box with pure Python 2 modules
   - integration possible with C extensions using 
 - Speed
   - not so great with short run processes or places where code spends most of its time in C
 - Memory
 - Stackless
 - Python 3 support in beta (yet)

In [34]:
%%time
%%pypy

print sum(xrange(10**9))

499999999500000000
CPU times: user 4.77 ms, sys: 13 ms, total: 17.8 ms
Wall time: 2.19 s


In [35]:
%%time
print sum(xrange(10 ** 9))

499999999500000000
CPU times: user 18.1 s, sys: 23.9 ms, total: 18.1 s
Wall time: 18.2 s


### `numba`

 - jit compiles python code to native machine instructions
 - utilizes the LLVM infrastructure to generate efficient machine code
 - can be configured to work on both CPUs, GPUs

#### using `numpy`
```python
import numpy as np

def pairwise_python(X, D):
    M = X.shape[0]
    N = X.shape[1]
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)

X, D = np.random.random((1000, 3)), np.empty((1000, 1000))
%timeit pairwise_python(X, D)
1 loops, best of 3: 12.1 s per loop
```

#### using `numba`
```python
import numpy as np
from numba import double
from numba.decorators import jit

@jit(arg_types=[double[:,:], double[:,:]])
def pairwise_numba(X, D):
    M = X.shape[0]
    N = X.shape[1]
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)

X, D = np.random.random((1000, 3)), np.empty((1000, 1000))
%timeit pairwise_python(X, D)
100 loops, best of 3: 15.5 ms per loop
```

**Reference:** https://jakevdp.github.io/blog/2012/08/24/numba-vs-cython/

### Nuitka
 - Python compiler compatible with Python 2.6, 2.7, 3.2, 3.3, and 3.4
 - Almost 2x faster than CPython
 - Builds a single executable

### Things not covered

 - Cython
 - Numpy / Pandas / scientific Python stack
 - asyncio
 - Greenlets, gevent
 - Stackless Python
 - Pyston

### Other possible optimizations

 - String concatenation is better with `''.join(seq) [O(n)]` instead of `+` or `+=` can be `[O(n**2)]` process because new strings may be built for each intermediate step.
 - `x = 10; y = 20` over `x, y = 10, 20`
 - `x < y < z` over `x < y and y < z`
 - `map` over simple `for` loops

## and one more thing ...

### only optimize when you **have** to

### "Premature optimization is the root of all evil"

and yes

# Questions?

#### Dhruv Baldawa (@dhruvbaldawa)

# References

 1. https://wiki.python.org/moin/PythonSpeed
 2. https://wiki.python.org/moin/TimeComplexity
 3. https://wiki.python.org/moin/PythonImplementations
 4. https://wiki.python.org/moin/PythonSpeed/PerformanceTips
 5. http://pypy.org/performance.html
 6. http://www.udacity.com/wiki/CS215%20Unit%201%20Code?course=cs215
 7. http://justindailey.blogspot.in/2011/09/python‐range-vs‐xrange.html
 8. http://en.wikipedia.org/wiki/List_comprehension
 9. http://docs.python.org/library/collections.html
 10. http://en.wikipedia.org/wiki/Memoization
 11. http://en.wikipedia.org/wiki/Collatz_conjecture